<a href="https://colab.research.google.com/github/anujgupta82/NLP_Bootcamp/blob/V_2_0/module4/Introduction%20to%20Character%20Embeddings%20(tf).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.11-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.11-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.11-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redir

------

LEFTOVERS:   


    1) Tensor board graphs for loss    
    2) Tensor board for network vis   
    3) Is embedding good or not ?
    4) Embedding visualization ?
    5) model perplexity
    
------

In [0]:
%matplotlib inline

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os.path

In [0]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='Untitled3_1_LOG.txt', mode='a')
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

## Data conversation utils

In [5]:
data_file = '/content/drive/NLP_bootcamp/Data/harry_potter_3.txt'

fp = open(data_file, 'r')
data = fp.read()

print (type(data))

print (data[:200])

<class 'str'>
Harry Potter and the Prisoner of Azkaban

by J.K. Rowling

CHAPTER ONE

OWL POST

Harry Potter was a highly unusual boy in many ways. For one thing, he
hated the summer holidays more than any other ti


In [6]:
#get the character set of data
chars = list(set(data))
vocab_size = len(chars)

print (len(chars), chars)

79 ['n', 'H', 'I', 'm', '3', 'z', 'r', ' ', 'k', 'w', 'T', '-', 'Y', 'G', 'J', 'j', '0', 'o', '?', 'B', "'", '!', 'a', '1', 'f', 'y', '7', 't', ';', '\\', '`', '\n', 'K', 'O', 'P', 'W', 'F', 'v', '&', '9', 'l', 'Q', '"', 'g', 'q', 'i', 'E', 's', 'b', 'S', 'p', 'x', 'L', 'A', 'd', '.', 'D', '6', '4', 'e', ',', 'M', '*', 'N', 'V', '_', 'u', 'X', ':', 'h', 'R', '2', 'Z', 'c', 'U', '5', ')', '(', 'C']


In [7]:
data_size, char_set_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, char_set_size))


data has 611606 characters, 79 unique.


In [0]:
char_to_ix = {ch:i for i,ch in enumerate(chars)}

In [0]:
ix_to_char = {i:ch for i,ch in enumerate(chars)}

In [10]:
input = 'ABCDabcd'
map(lambda ch: char_to_ix[ch], input)  

In [0]:
def c2i(input):
    return map(lambda ch: char_to_ix[ch], input)  

In [0]:
def i2c(input):
    return map(lambda ch: ix_to_char[ch], input)  

--------- 

## Prepare Training data
--------- 

raw_data = 'abcdefghijklmonpqrstuvwxyzABCDEGHIJKLMONPQRSTUVWXYZ123456789!@#$%^&*()'

data_len = len(raw_data)
print data_len

seq_length = 3 # number of steps to unroll the RNN for
batch_size = 6

num_examples = (data_len - 1) // seq_length
print num_examples

num_batches = num_examples // batch_size
print num_batches

for i in range(num_batches):
    print "i = %d" %i
    idx = i * batch_size * seq_length
    print "idx = %d" %idx
    
    for j in range(batch_size):
        start_index = idx + (j*seq_length)
        end_index = idx + (j+1)*seq_length
        print "start_index = %d, end_index = %d" %(start_index, end_index)

raw_data.upper()

In [0]:
# Generate data for an epoch, with batches of size batch_size.

def gen_epoch_data(raw_data, batch_size, seq_length=25):
    
    data_len = len(raw_data)
    
    # num_examples is num_of_sequences. each sequence is an example
    num_examples = (data_len - 1) // seq_length
    
    num_batches = num_examples // batch_size 
    
    # intuituion for the abovce 2 lines is - we are to feed sequences. If each seq is of length k and there are N 
    # sequences in each batch - then a batch basically consists of k*N characters; 
    # thus number of batches = data_len // (seq_lenth * batch_size)
    
    # An easier way to see this is : num of data points = data_len // seq_length
    # therefore num of batches = num of data points // batch_size
    
    # to get training data, we iterate over batches/chunks/rows. First we establish from where the batch data starts 
    # imagine data to be a a st line ------------- of length data_len
    # we split it into batches 
    #    ----------------
    #    ----------------
    #    ----------------
    
    # These (^) text split into chunks/rows - num_of_chunks =  data_len // (seq_lenth * batch_size)
    # first get which chuck to pick. This is done by idx = i * batch_size * seq_length
    
    # once u have the right row - each point in batch is a seq,seq, we need to pick num_of_piars = batch_size
    # so iterate over batch_size
    # starting point in a chunk is jth sequence * length of 1 seq
    
    epoch_data = []
    
    for i in range(num_batches):
        batch = []
        idx = i * batch_size * seq_length
        
        for j in range(batch_size):
            start_index = idx + (j*seq_length)
            end_index = idx + (j+1)*seq_length
            input_ = raw_data[start_index : end_index]
            target_ = raw_data[start_index + 1 : end_index +1 ]
            
            batch.append([c2i(input_), c2i(target_)])
            
        epoch_data.append(batch)
        
    return epoch_data

# Test Data Generation 

In [0]:
check_data = gen_epoch_data(raw_data=data, batch_size=32)

In [15]:
print (type(check_data), len(check_data))
print (len(check_data[0]))
print (check_data[0]) # this is a list of lists of lists

<class 'list'> 764
32
[[<map object at 0x7f19a9bafac8>, <map object at 0x7f19a9bafb38>], [<map object at 0x7f19a9bafba8>, <map object at 0x7f19a9bafc18>], [<map object at 0x7f19a9bafc88>, <map object at 0x7f19a9bafcf8>], [<map object at 0x7f19a9bafd68>, <map object at 0x7f19a9bafdd8>], [<map object at 0x7f19a9bafe48>, <map object at 0x7f19a9bafeb8>], [<map object at 0x7f19a9baff28>, <map object at 0x7f19a9baff98>], [<map object at 0x7f19a9bac048>, <map object at 0x7f19a9bac0b8>], [<map object at 0x7f19a9bac128>, <map object at 0x7f19a9bac198>], [<map object at 0x7f19a9bac208>, <map object at 0x7f19a9bac278>], [<map object at 0x7f19a9bac2e8>, <map object at 0x7f19a9bac358>], [<map object at 0x7f19a9bac3c8>, <map object at 0x7f19a9bac438>], [<map object at 0x7f19a9bac4a8>, <map object at 0x7f19a9bac518>], [<map object at 0x7f19a9bac588>, <map object at 0x7f19a9bac5f8>], [<map object at 0x7f19a9bac668>, <map object at 0x7f19a9bac6d8>], [<map object at 0x7f19a9bac748>, <map object at 0x7f1

# Set Params

In [0]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 2e-1
batch_size = 50


#num_epochs = 500
num_epochs = 1

# Build the Network

In [0]:
import tensorflow as tf

# Input
x = tf.placeholder(tf.int32, shape=(seq_length), name="x")
y = tf.placeholder(tf.int32, shape=(seq_length), name="y")


In [0]:
# One Hot representation of the input
x_oh = tf.one_hot(indices=x, depth=vocab_size)
y_oh = tf.one_hot(indices=y, depth=vocab_size)

In [19]:
#check and balances 

print (x.get_shape())
print (y.get_shape())

print (x_oh.get_shape())
print (y_oh.get_shape())

(25,)
(25,)
(25, 79)
(25, 79)


In [0]:
# input and output as of now is 25 characters, each in 1-hot format of length 80
# we will need to feed it 1 character (in 1-hot format) at a time   

#We will convert it into desired format in 2 steps - from (25,80) to 25 tensors of
#shape (80,) then to 25 tensors of shape (1,80)

# Convert the VxN tensor into list of V tensors each of shape = (V,).
rnn_inputs = tf.unstack(value=x_oh)
rnn_targets = tf.unstack(value=y_oh)

#Unpacks the given dimension of a rank=R tensor into rank=(R-1) tensors
#i'th tensor in output is the slice value[i, :]. 
# given a tensor of shape (A, B, C, D);
#If axis == 0 (default value) then the i'th tensor in output is the slice value[i, :, :, :] 
#and return a list of tensors, where each tensor in output will have shape (B, C, D)

# for our data, rank of x_oh is 2 and shape = (25,80)
#rank is number of dimensions


In [21]:
rnn_inputs[0].get_shape()

TensorShape([Dimension(79)])

In [0]:
# Setup the weights and biases.
with tf.variable_scope('rnn_cell'):
    
    Wxh = tf.Variable(tf.truncated_normal(shape=(hidden_size, vocab_size)), name='Wxh')
    Whh = tf.Variable(tf.truncated_normal(shape=(hidden_size, hidden_size)), name='Whh')
    Why = tf.Variable(tf.truncated_normal(shape=(vocab_size, hidden_size)), name='Why')    
    bh = tf.Variable(tf.zeros(shape=(hidden_size, 1)), name='bh')
    by = tf.Variable(tf.zeros(shape=(vocab_size,1)),name='by')

In [0]:

def rnn_cell(rnn_input, cur_state):
    
    # expand_dims is required to make the input a 2-D tensor.
    # Given a tensor input, this operation inserts a dimension of 1 at the dimension 
    # index axis of input's shape. The dimension index axis starts at zero;
    
    with tf.variable_scope('rnn_cell'):
        
        inp = tf.expand_dims(input=rnn_input, axis=1)

        next_state = tf.tanh(tf.matmul(Wxh, inp) + tf.matmul(Whh, cur_state) + bh)
        y_hat = tf.matmul(Why, next_state) + by
    
    return y_hat, next_state

In [24]:
by.get_shape()

TensorShape([Dimension(79), Dimension(1)])

In [0]:
logits = []

#initialise state
state = tf.zeros([hidden_size, 1])

# Iterate over all the input vectors
for rnn_input in rnn_inputs:
    y_hat, state = rnn_cell(rnn_input, state)
    # Convert y_hat into a 1-D tensor
    y_hat = tf.squeeze(y_hat)
    logits.append(y_hat)

In [0]:
# Use the helper method to compute the softmax losses
# (It basically compares the outputs to the expected output)
for logit, target in zip(logits, rnn_targets):
    losses = [tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=target)] 
              
# Compute the average loss over the batch
total_loss = tf.reduce_mean(losses)

In [0]:
# Under the hood, the operation below computes the gradients and does the backprop!
optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(total_loss)



In [0]:
init_state = tf.zeros([hidden_size, 1])
sample_state = init_state

seed = tf.placeholder(tf.int32, [1], name='seed')
rnn_input = tf.one_hot(seed, vocab_size)
rnn_input = tf.squeeze(rnn_input)

y_hat, sample_state = rnn_cell(rnn_input, sample_state)

prob = tf.nn.softmax(tf.squeeze(y_hat)) 

# Network defined, lets train it

In [0]:

epoch_data = gen_epoch_data(data, batch_size)

tlosses = []

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter('./Char_Model.log', sess.graph)
    
    for epoch_idx in range(num_epochs):
        
        print ('--- Starting Epoch:', epoch_idx, '---')
        epoch_loss = 0
        #epoch_state = np.zeros([hidden_size, 1])
        equals = 0.0
        
        for idx, batch in enumerate(epoch_data):
            
            training_loss = 0
            for example_idx, example in enumerate(batch):
                
                x_i = example[0]
                y_i = example[1]
                x_i_val = np.array(list(x_i))
                y_i_val = np.array(list(y_i))
                
                #loss, tloss, _, logits_, rnn_targets_, epoch_state = \
                #            sess.run([losses, total_loss, train_step, logits, rnn_targets, state], \
                #            feed_dict={x:x_i, y:y_i})
                
                loss, tloss, _, logits_, rnn_targets_ = \
                            sess.run([losses, total_loss, optimizer, logits, rnn_targets], \
                            feed_dict={x:x_i_val, y:y_i_val})
                    
                    
                    
                _, tloss, _, _  = sess.run([optimizer, total_loss, logits, rnn_targets], \
                            feed_dict={x:x_i_val, y:y_i_val})
                print(tloss)
                    
                    
                logits_argmax = np.argmax(logits_, axis=1)
                rnn_targets_argmax = np.argmax(rnn_targets_, axis=1)
                equals += np.sum(logits_argmax == rnn_targets_argmax)
                training_loss += tloss
                
                if (example_idx % 500 == 0):
                    if list(example[0]) != []:
                        inp_seed = np.array([list(example[0])])
                    else:
                        inp_seed = np.array([1])
                    #print(inp_seed.shape)
                    #print '\n'
                    #print '--- SAMPLE BEGIN ---'
                    
                    logging.debug("\n")
                    logging.debug("--- SAMPLE BEGIN ---")
                    
                    num_chars = 100
                    ixes = []
                    sstate = np.zeros([hidden_size, 1])
                    for j in range(num_chars):
                        prob_r, sstate = sess.run([prob, sample_state], feed_dict={seed:inp_seed, init_state:sstate, x:x_i_val})
                        ix = np.random.choice(vocab_size, p=prob_r.ravel())
                        ixes.append(ix)
                        inp_seed = np.array([ix])

                    #print ''.join(i2c(ixes))
                    #print '--- SAMPLE END ---\n'
                    sent = ''.join(i2c(ixes))
                    logging.debug(sent)
                    logging.debug("--- SAMPLE END ---\n")
                
            training_loss /= len(batch)
            equals /= len(batch)
            print ('Epoch:', epoch_idx, 'Batch:', idx)
            print ('Average training loss in batch:', training_loss)
            print ('Average matching chars per batch:', equals)
            tlosses.append(training_loss)
                

--- Starting Epoch: 0 ---
13.915478
15.954452
22.299332
24.767673
12.958686
10.379598
31.2315
19.467064
30.287954
19.864029
25.84103
42.0401
13.339578
26.763142
7.305488
20.884024
18.479773
22.673935
32.654232
29.495197
31.625408
21.25359
13.870102
33.26302
20.924986
30.421436
26.02698
19.87274
17.268356
9.028865
37.918728
28.751934
19.966488
9.598641
25.841673
18.235792
26.89827
29.136314
28.588469
7.938715
4.9402337
33.76788
30.478893
13.436449
38.331467
20.10297
10.853843
35.299442
15.807275
26.867111
Epoch: 0 Batch: 0
Average training loss in batch: 22.538366661071777
Average matching chars per batch: 0.22
23.705711
5.7545614
40.715565
6.1003895
10.276669
21.25372
16.169895
1.7807982
28.114586
20.976141
22.255445
27.766623
17.190363
25.625134
24.778862
23.273245
18.18209
27.05605
27.841316
39.07007
17.529707
18.651548
32.006443
15.190421
18.074682
30.681038
19.901152
23.63875
22.516792
16.015635
15.781042
22.811214
30.52573
31.919731
19.72971
30.128613
21.371359
32.34261
28.044537


In [0]:
#tlosses = train()
plt.plot(tlosses)
plt.show()



In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os.path

data_file= '/Users/admin/Documents/Anuj/Coding/Warehouse/input.txt'

data = open(data_file, 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 2e-1
batch_size = 50
num_epochs = 500

In [0]:
# Convert an array of chars to array of vocab indices
def c2i(inp):
    return map(lambda c:char_to_ix[c], inp)

def i2c(inp):
    return map(lambda c:ix_to_char[c], inp)

# Generate data for an epoch, with batches of size batch_size.
def gen_epoch_data(raw_data, batch_size):
    data_len = len(raw_data)
    num_examples = (data_len - 1) // seq_length
    num_batches = num_examples // batch_size

    epoch_data = []
    for i in range(num_batches):
        batch = []
        idx = i * batch_size * seq_length
        for j in range(batch_size):
            inp = raw_data[idx + j*seq_length:idx + (j+1)*seq_length]
            target = raw_data[idx + 1+(j*seq_length):idx + 1+((j+1)*seq_length)]

            batch.append([c2i(inp), c2i(target)])
        epoch_data.append(batch)
    return epoch_data


In [0]:
epoch_data = gen_epoch_data(data, batch_size)
init_state = tf.zeros([hidden_size, 1])

# Input
x = tf.placeholder(tf.int32, shape=(seq_length), name="x")
y = tf.placeholder(tf.int32, shape=(seq_length), name="y")
state = tf.zeros([hidden_size, 1])

# One Hot representation of the input
x_oh = tf.one_hot(indices=x, depth=vocab_size)
y_oh = tf.one_hot(indices=y, depth=vocab_size)

rnn_inputs = tf.unstack(x_oh)
rnn_targets = tf.unstack(y_oh)


In [0]:
# Setup the weights and biases.
with tf.variable_scope('rnn_cell'):
    Wxh = tf.get_variable('Wxh', [hidden_size, vocab_size])
    Whh = tf.get_variable('Whh', [hidden_size, hidden_size])
    Why = tf.get_variable('Why', [vocab_size, hidden_size])
    bh = tf.get_variable('bh', [hidden_size, 1])
    by = tf.get_variable('by', [vocab_size, 1])

# Actual math behind computing the output and the next state of the RNN.
def rnn_cell(rnn_input, cur_state):
    with tf.variable_scope('rnn_cell', reuse=True):
        Wxh = tf.get_variable('Wxh', [hidden_size, vocab_size])
        Whh = tf.get_variable('Whh', [hidden_size, hidden_size])
        Why = tf.get_variable('Why', [vocab_size, hidden_size])
        bh = tf.get_variable('bh', [hidden_size, 1])
        by = tf.get_variable('by', [vocab_size, 1])
    inp = tf.expand_dims(rnn_input, 1)

    next_state = tf.tanh(tf.matmul(Wxh, inp) + tf.matmul(Whh, cur_state) + bh)
    y_hat = tf.matmul(Why, next_state) + by
    return y_hat, next_state

In [0]:
logits = []
for rnn_input in rnn_inputs:
    y_hat, state = rnn_cell(rnn_input, state)
    y_hat = tf.squeeze(y_hat)
    logits.append(y_hat)

losses = [tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=target) for logit, target in zip(logits, rnn_targets)]
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(total_loss)

num_samples = 25
sample_state = init_state
seed = tf.placeholder(tf.int32, [1], name='seed')
rnn_input = tf.one_hot(seed, vocab_size)
ixes = []

rnn_input = tf.squeeze(rnn_input)
y_hat, sample_state = rnn_cell(rnn_input, sample_state)
prob = tf.nn.softmax(tf.squeeze(y_hat))



In [0]:
def train():
    
    tlosses = []
    
    #save the model
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        if os.path.isfile("model.ckpt"):
            saver.restore(sess, "model.ckpt")
        else:
            sess.run(tf.global_variables_initializer())

        for epoch_idx in range(num_epochs):
            print ('--- Starting Epoch:', epoch_idx, '---')
            epoch_loss = 0
            epoch_state = np.zeros([hidden_size, 1])
            equals = 0.0
            for idx, batch in enumerate(epoch_data):
                training_loss = 0
                for example_idx, example in enumerate(batch):
                    x_i = example[0]
                    y_i = example[1]
                    x_i_val = np.array(list(x_i))
                    y_i_val = np.array(list(y_i))
                    
                    loss, tloss, _, logits_, rnn_targets_, epoch_state = \
                        sess.run([losses, total_loss, train_step, logits, \
                            rnn_targets, state], \
                                feed_dict={x:x_i_val, y:y_i_val, init_state:epoch_state}
                        )

                    logits_argmax = np.argmax(logits_, axis=1)
                    rnn_targets_argmax = np.argmax(rnn_targets_, axis=1)
                    equals += np.sum(logits_argmax == rnn_targets_argmax)

                    training_loss += tloss

                    if (example_idx % 100 == 0):
                        if list(example[0]) != []:
                            inp_seed = np.array([list(example[0])])
                        else:
                            inp_seed = np.array([1])

                        print ('\n')
                        print ('--- SAMPLE BEGIN ---')
                        num_chars = 100
                        ixes = []
                        sstate = np.zeros([hidden_size, 1])
                        for j in range(num_chars):
                            prob_r, sstate = sess.run([prob, sample_state], feed_dict={seed:inp_seed, init_state:sstate, x:x_i_val})
                            ix = np.random.choice(vocab_size, p=prob_r.ravel())
                            ixes.append(ix)
                            inp_seed = np.array([ix])

                        print (''.join(i2c(ixes)))
                        print ('--- SAMPLE END ---')

                training_loss /= len(batch)
                equals /= len(batch)
                print ('Epoch:', epoch_idx, 'Batch:', idx)
                print ('Average training loss in batch:', training_loss)
                print ('Average matching chars per batch:', equals)
                tlosses.append(training_loss)
            save_path = saver.save(sess, "model.ckpt")
            print("Model saved in file: %s" % save_path)
    return tlosses


In [0]:
tlosses = train()
plt.plot(tlosses)
plt.show()